In [ ]:
from bs4 import BeautifulSoup 
from selenium import webdriver
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
from selenium.common.exceptions import NoSuchElementException
import keyboard
import regex as re

#Open Chrome browser
driver=webdriver.Chrome()
driver.get('https://deliveroo.fr/en/restaurants/paris/9eme-opera/?geohash=u09wj3sv9zm1&category=burgers')

accept_btn = driver.find_element(By.ID, 'onetrust-accept-btn-handler')
accept_btn.click()

ok_btn = driver.find_element(By.XPATH, "//span[text()='OK']")
ok_btn.click()

#Initialize workbook
workbook = Workbook()
worksheet = workbook.active

# Set the column headers
worksheet.cell(row=1, column=1, value="Restaurant Name")
worksheet.cell(row=1, column=2, value="Address")
worksheet.cell(row=1, column=3, value="Phone")
worksheet.cell(row=1, column=4, value="Link")

counter = 0 # Initialize the counter variable
index = 0

while True:
    buttons = driver.find_elements(By.CSS_SELECTOR, "p.ccl-649204f2a8e630fd.ccl-a396bc55704a9c8a.ccl-ff5caa8a6f2b96d0.ccl-40ad99f7b47f3781")
    while index < len(buttons):
        button = buttons[index]
        restaurant_name = button.text # Get restaurant name from button text
        button.click()
        time.sleep(3)
        link = driver.current_url
        info_btn = driver.find_element(By.XPATH, "//span[text()='Info']")
        info_btn.click()

        try:
            address_elem = driver.find_element(By.XPATH, "//span[contains(@class, 'ccl-0956b2f88e605eb8') and contains(text(), ', Paris, ')]")
            time.sleep(2)
            address = address_elem.text.strip()
        except NoSuchElementException:
            address = ""

        try:
            phone_elem = driver.find_element(By.XPATH, "//a[contains(@class, 'UIContentCard') and starts-with(@href, 'tel:')]/div/span[contains(@class, 'ccl')]")
            phone_text = phone_elem.text
            match = re.search(r'\+\d+', phone_text)
            phone = match.group() if match else ""
        except NoSuchElementException:
            phone = ""

        keyboard.press_and_release('esc')

        driver.back()
        worksheet.append([restaurant_name, address, phone, link]) # Write the restaurant name, address, phone and link to the worksheet

        buttons = driver.find_elements(By.CSS_SELECTOR, "p.ccl-649204f2a8e630fd.ccl-a396bc55704a9c8a.ccl-ff5caa8a6f2b96d0.ccl-40ad99f7b47f3781")
        index+=1
        
    # Get the last element of the current page
    last_element = buttons[-1]
    
    # Scroll down to the last element of the current page
    driver.execute_script("arguments[0].scrollIntoView();", last_element)
       
# Save the workbook
workbook.save("restaurant_info.xlsx")